In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import re

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=10000,stop_words='english')

In [3]:
import pickle

In [3]:
employee = pd.read_csv("../input/employee.csv")

In [9]:
employee.shape

(829, 20)

In [10]:
employee.head()

,_id,commutation,date_of_birth,date_of_joining,desk,email,employee_id,first_name,id_card_no,image_url,last_name,status,department_id,designation_id,du_head_id,manager_id,coe,category,date_of_confirmation,location
0,395,PERSONAL,2021-02-07T18:30Z,2021-01-24T18:30Z,NaN,sweety.narendra@afourtech.com,AFTP003,Sweety Alwin,NaN,NaN,Narendra,ACTIVE,3,76.0,NaN,NaN,NONE,CONTRACT,2021-01-24T18:30Z,OFFSHORE
1,10,PERSONAL,1991-03-31T00:00:00.09Z,2015-01-15T00:00:00.09Z,NaN,kiran.argade@afourtech.com,AFT00308,Kiran,NaN,NaN,Argade,ACTIVE,2,115.0,NaN,NaN,NONE,NaN,NaN,NaN
2,11,PERSONAL,1991-10-09T00:00:00.09Z,2016-01-02T00:00:00.09Z,NaN,dipesh.mali@afourtech.com,AFT00342,Dipesh,NaN,NaN,Mali,ACTIVE,1,95.0,NaN,NaN,NONE,NaN,NaN,NaN
3,93,PERSONAL,1994-10-29T00:00+05:30,2017-03-26T00:00+05:30,NaN,shubham.jain@afourtech.com,AFT00416,Shubham,NaN,NaN,Jain,INACTIVE,3,11.0,NaN,NaN,DESIGN_DEVELOPMENT,EMPLOYEE,2021-07-29T00:00+05:30,OFFSHORE
4,417,PERSONAL,1994-07-27T18:30Z,2021-03-08T18:30Z,NaN,supriya.barkund@afourtech.com,AFT00782,Supriya,NaN,NaN,Barkund,ACTIVE,3,11.0,NaN,NaN,DESIGN_DEVELOPMENT,EMPLOYEE,2021-03-08T18:30Z,OFFSHORE


In [11]:
project = pd.read_csv("../input/project_202210171657.csv")

In [12]:
department = pd.read_csv("../input/department_202210171655.csv") 
designation = pd.read_csv("../input/designation_202210171655.csv") 

In [13]:
employee = pd.merge(employee, department, on='department_id', how='outer')

In [14]:
result = employee.dtypes
print(result)

_id_x                   float64
commutation              object
date_of_birth            object
date_of_joining          object
desk                    float64
email                    object
employee_id              object
first_name               object
id_card_no               object
image_url               float64
last_name                object
status                   object
department_id             int64
designation_id          float64
du_head_id              float64
manager_id              float64
coe                      object
category                 object
date_of_confirmation     object
location                 object
_id_y                     int64
department_name          object
dtype: object


In [15]:
#employee1['designation_id'] = employee1['designation_id'].astype(int)

In [16]:
employee = pd.merge(employee, designation, on='designation_id', how='left')

In [17]:
employee.isnull().sum()

_id_x                     1
commutation               2
date_of_birth             2
date_of_joining           1
desk                    832
email                     1
employee_id               1
first_name                1
id_card_no              829
image_url               832
last_name                 1
status                    1
department_id             0
designation_id            2
du_head_id              832
manager_id              832
coe                       1
category                223
date_of_confirmation    223
location                223
_id_y                     0
department_name           0
_id                       2
designation_name          2
dtype: int64

In [18]:
employee.shape

(832, 24)

In [19]:
employee.columns

Index(['_id_x', 'commutation', 'date_of_birth', 'date_of_joining', 'desk',
       'email', 'employee_id', 'first_name', 'id_card_no', 'image_url',
       'last_name', 'status', 'department_id', 'designation_id', 'du_head_id',
       'manager_id', 'coe', 'category', 'date_of_confirmation', 'location',
       '_id_y', 'department_name', '_id', 'designation_name'],
      dtype='object')

In [20]:
employee['employee_id'].unique().shape

(830,)

In [21]:
employee.drop(columns=['_id_x', 'commutation', 'date_of_birth', 'date_of_joining', 'desk',
       'email', 'id_card_no', 'image_url',
        'status', 'department_id', 'designation_id', 'du_head_id',
       'manager_id', 'coe', 'date_of_confirmation', 'location',
       '_id_y', '_id'], axis=1, inplace=True)

In [22]:
#employee.drop(employee.columns[[0,1,2,3,4,5,8,9,11,12,13,14,15,16,18,19,20,22]], axis=1, inplace=True)

In [23]:
employee.category = np.where(employee.employee_id.str.startswith("AFT0"), employee.category.ffill(), employee.category)

In [24]:
employee.category = np.where(employee.employee_id.str.startswith("AFTP"), employee.category.ffill(), employee.category)

In [25]:
employee.category = np.where(employee.employee_id.str.startswith("AFTI"), employee.category.ffill(), employee.category)

In [26]:
employee["category"].fillna("EMPLOYEE", inplace = True)

In [27]:
employee["category"].value_counts()

EMPLOYEE     554
INTERN       153
CONTRACT     116
PROBATION      9
Name: category, dtype: int64

In [28]:
employee.isnull().sum()

employee_id         1
first_name          1
last_name           1
category            0
department_name     0
designation_name    2
dtype: int64

In [29]:
employee.shape

(832, 6)

In [30]:
employee = employee.dropna(axis=0, subset=['designation_name'])

In [31]:
employee = employee.dropna(axis=0, subset=['department_name'])

In [33]:
project_resource = pd.read_csv("../input/employee_project_resource_202210171656.csv")

In [34]:
project_resource.columns

Index(['_id', 'employee_id', 'project_id', 'project_resource_id', 'is_active',
       'allocation_percentage', 'reporting_manager', 'start_date', 'end_date',
       'duhead_id', 'coehead_id'],
      dtype='object')

In [35]:
project.columns

Index(['project_id', 'actual_end_date', 'actual_start_date', 'coe',
       'cost_center', 'description', 'engagement_type', 'expected_end_date',
       'expected_start_date', 'name', 'services_type', 'status',
       'sub_services_type', 'work_location', 'project_type', 'service_type'],
      dtype='object')

In [36]:
project_resource.drop(columns=['_id', 'project_resource_id', 'is_active',
       'reporting_manager', 'start_date', 'end_date',
       ], axis=1, inplace=True)

project.drop(columns=['actual_end_date', 'actual_start_date',
       'cost_center', 'description', 'engagement_type', 'expected_end_date',
       'expected_start_date', 'services_type', 'status',
       'sub_services_type', 'service_type'], axis=1, inplace=True)

In [37]:
#project_resource.drop(project_resource.columns[[0,3,4,6,7,8]], axis=1, inplace=True)
#project.drop(project.columns[[1,2,4,5,6,7,8,10,11,12,15]], axis=1, inplace=True)

In [38]:
project.head()

,project_id,coe,name,work_location,project_type
0,92,RELIABILITY_ENGINEERING,Airtel Testing,OFFSHORE,CUSTOMER
1,42,DESIGN_DEVELOPMENT,iLink BI,OFFSHORE,CUSTOMER
2,1,RELIABILITY_ENGINEERING,A4 Services,OFFSHORE,ASSET
3,64,DESIGN_DEVELOPMENT,Engage Marketing Inc.,OFFSHORE,CUSTOMER
4,69,RELIABILITY_ENGINEERING,Kristal AI - QA,OFFSHORE,CUSTOMER


In [39]:
project_resource.head()

,employee_id,project_id,allocation_percentage,duhead_id,coehead_id
0,AFT00226,14,50,AFT00129,AFT00640
1,AFTC0198,28,0,NaN,NaN
2,AFTC0198,28,0,NaN,NaN
3,AFTC0198,28,0,NaN,NaN
4,AFTC0198,28,0,NaN,NaN


In [40]:
project = pd.merge(project_resource, project, on='project_id', how='left')

In [41]:
project.head()

,employee_id,project_id,allocation_percentage,duhead_id,coehead_id,coe,name,work_location,project_type
0,AFT00226,14,50,AFT00129,AFT00640,RELIABILITY_ENGINEERING,IBH-Dev,ONSITE_OFFSHORE,CUSTOMER
1,AFTC0198,28,0,NaN,NaN,DESIGN_DEVELOPMENT,Marchex (Sustenance),ONSITE,CUSTOMER
2,AFTC0198,28,0,NaN,NaN,DESIGN_DEVELOPMENT,Marchex (Sustenance),ONSITE,CUSTOMER
3,AFTC0198,28,0,NaN,NaN,DESIGN_DEVELOPMENT,Marchex (Sustenance),ONSITE,CUSTOMER
4,AFTC0198,28,0,NaN,NaN,DESIGN_DEVELOPMENT,Marchex (Sustenance),ONSITE,CUSTOMER


In [42]:
employee_project = pd.merge(employee,project, on='employee_id', how='left')

In [43]:
employee_project.head()

,employee_id,first_name,last_name,category,department_name,designation_name,project_id,allocation_percentage,duhead_id,coehead_id,coe,name,work_location,project_type
0,AFTP003,Sweety Alwin,Narendra,CONTRACT,Engineering,Consultant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFT00416,Shubham,Jain,EMPLOYEE,Engineering,Senior Software Development Engineer,7.0,100.0,AFT00599,AFT00599,RELIABILITY_ENGINEERING,Bench,OFFSHORE,ASSET
2,AFT00416,Shubham,Jain,EMPLOYEE,Engineering,Senior Software Development Engineer,89.0,100.0,AFT00599,AFT00599,DESIGN_DEVELOPMENT,VMS,OFFSHORE,ASSET
3,AFT00416,Shubham,Jain,EMPLOYEE,Engineering,Senior Software Development Engineer,1.0,100.0,AFT00599,AFT00599,RELIABILITY_ENGINEERING,A4 Services,OFFSHORE,ASSET
4,AFT00416,Shubham,Jain,EMPLOYEE,Engineering,Senior Software Development Engineer,109.0,100.0,AFT00640,AFT00599,DESIGN_DEVELOPMENT,ABS Development,OFFSHORE,CUSTOMER


In [44]:
employee_project.drop(columns=['project_id'], axis=1, inplace=True)

In [45]:
employee_project.isnull().sum()

employee_id                0
first_name                 0
last_name                  0
category                   0
department_name            0
designation_name           0
allocation_percentage     43
duhead_id                248
coehead_id               277
coe                       43
name                      43
work_location             43
project_type              43
dtype: int64

In [46]:
employee_project.shape


(2874, 13)

In [47]:
employee_project = employee_project.drop_duplicates(
  subset = ['employee_id', 'name'],
  keep = 'last').reset_index(drop = True)

In [48]:
df_headids = employee_project[['employee_id', 'duhead_id']].copy()
df_headids['duhead_id'].fillna(df_headids['duhead_id'].mode()[0], inplace=True)
df_headids.drop_duplicates(inplace=True)
df_headids = df_headids.groupby('employee_id')['duhead_id'].apply(list).reset_index(name='duhead_id')
print(df_headids.shape)
df_headids.isnull().sum()

(828, 2)


employee_id    0
duhead_id      0
dtype: int64

In [49]:
employee = pd.merge(employee, df_headids, on='employee_id', how='left')

In [50]:
df_headids = employee_project[['employee_id', 'coehead_id']].copy()
df_headids['coehead_id'].fillna(df_headids['coehead_id'].mode()[0], inplace=True)
df_headids.drop_duplicates(inplace=True)
df_headids = df_headids.groupby('employee_id')['coehead_id'].apply(list).reset_index(name='coehead_id')
print(df_headids.shape)
df_headids.isnull().sum()

(828, 2)


employee_id    0
coehead_id     0
dtype: int64

In [51]:
employee = pd.merge(employee, df_headids, on='employee_id', how='left')

In [52]:
employee.columns

Index(['employee_id', 'first_name', 'last_name', 'category', 'department_name',
       'designation_name', 'duhead_id', 'coehead_id'],
      dtype='object')

In [53]:
del df_headids

In [54]:
df_coe = employee_project[['employee_id', 'coe']].copy()
df_coe['coe'].fillna(df_coe['coe'].mode()[0], inplace=True)
df_coe.drop_duplicates(inplace=True)
df_coe = df_coe.groupby('employee_id')['coe'].apply(list).reset_index(name='coe')
df_coe.shape
df_coe.isnull().sum()

employee_id    0
coe            0
dtype: int64

In [55]:
df_coe.shape

(828, 2)

In [56]:
employee = pd.merge(employee, df_coe, on='employee_id', how='left')

In [57]:
df_name = employee_project[['employee_id', 'name']].copy()
df_name["name"].fillna("Bench", inplace=True)
df_name.drop_duplicates(inplace=True)
df_name = df_name.groupby('employee_id')['name'].apply(list).reset_index(name='name')
print(df_name.shape)
df_name.isnull().sum()

(828, 2)


employee_id    0
name           0
dtype: int64

In [58]:
employee = pd.merge(employee, df_name, on='employee_id', how='left')

In [59]:
df_WL = employee_project[['employee_id', 'work_location']].copy()
df_WL['work_location'].fillna(df_WL['work_location'].mode()[0], inplace=True)
df_WL.drop_duplicates(inplace=True)
df_WL = df_WL.groupby('employee_id')['work_location'].apply(list).reset_index(name='work_location')
print(df_WL.shape)
df_WL.isnull().sum()

(828, 2)


employee_id      0
work_location    0
dtype: int64

In [60]:
employee = pd.merge(employee, df_WL, on='employee_id', how='left')

In [61]:
df_pt = employee_project[['employee_id', 'project_type']].copy()
df_pt['project_type'].fillna(df_pt['project_type'].mode()[0], inplace=True)
df_pt.drop_duplicates(inplace=True)
df_pt = df_pt.groupby('employee_id')['project_type'].apply(list).reset_index(name='project_type')
df_pt.shape

(828, 2)

In [62]:
employee = pd.merge(employee, df_pt, on='employee_id', how='left')

In [63]:
df_all = project_resource[['employee_id', 'allocation_percentage']].copy()
df_all['allocation_percentage'] = df_all['allocation_percentage'].fillna(0)
df_all['allocation_percentage'] = df_all['allocation_percentage'].astype(int)
df_all1 = df_all.groupby('employee_id', as_index =False)['allocation_percentage'].sum()
df_all1.shape

(790, 2)

In [64]:
employee = pd.merge(employee, df_all1, on='employee_id', how='left')

In [65]:
employee['allocation_percentage'] = employee['allocation_percentage'].fillna(0)

In [66]:
employee['allocation_percentage'] = employee['allocation_percentage'].astype(int)

In [67]:
employee.head()

,employee_id,first_name,last_name,category,department_name,designation_name,duhead_id,coehead_id,coe,name,work_location,project_type,allocation_percentage
0,AFTP003,Sweety Alwin,Narendra,CONTRACT,Engineering,Consultant,[AFT00304],[AFT00640],[DESIGN_DEVELOPMENT],[Bench],[OFFSHORE],[CUSTOMER],0
1,AFT00416,Shubham,Jain,EMPLOYEE,Engineering,Senior Software Development Engineer,"[AFT00599, AFT00640, AFT00304]",[AFT00599],"[RELIABILITY_ENGINEERING, DESIGN_DEVELOPMENT]","[Bench, VMS, A4 Services, ABS Development, MIS...",[OFFSHORE],"[ASSET, CUSTOMER]",800
2,AFT00782,Supriya,Barkund,EMPLOYEE,Engineering,Senior Software Development Engineer,"[AFT00628, AFT00640]","[AFT00599, AFT00640]",[DESIGN_DEVELOPMENT],"[Harmonic, MindStreet, Convoy- New, Convoy – S2]",[OFFSHORE],[CUSTOMER],400
3,AFT00455,Vipul,Jadhav,EMPLOYEE,Engineering,Senior Software Development Engineer,"[AFT00599, AFT00628]","[AFT00599, AFT00628]",[DESIGN_DEVELOPMENT],"[Marchex (Media Team), Marchex – Archenia]",[OFFSHORE],[CUSTOMER],300
4,AFT00599,Abhijit,Nadgonda,EMPLOYEE,Engineering,Director - Emerging Technologies,[AFT00001],"[AFT00640, AFT00001]","[DESIGN_DEVELOPMENT, RELIABILITY_ENGINEERING]","[Training Assessment Project, PWA, Airtel UX, ...",[OFFSHORE],"[ASSET, CUSTOMER]",100


In [68]:
employee_skills = pd.read_csv("../input/employee_skill_202210171656.csv")
skills =  pd.read_csv("../input/skills_202210171657.csv")

In [69]:
merged_employee_skills_and_skills = employee_skills.merge(skills, how='inner', left_on='skill_id', right_on='_id')

In [70]:
final_merged_employee_skills = merged_employee_skills_and_skills.drop(columns=['_id_x','_id_y','status'],axis=1)

In [71]:
final_merged_employee_skills = final_merged_employee_skills.replace(np.nan," ")

In [72]:
final_merged_employee_skills['employee_skills'] = final_merged_employee_skills['sub_technology']+" "+final_merged_employee_skills['technology']+" "+final_merged_employee_skills['tool']+" "+final_merged_employee_skills['work_type']

In [73]:
three_column_final_skills = final_merged_employee_skills.drop(columns = ['sub_technology', 'technology', 'tool',
       'work_type'],axis = 1)

In [74]:
grp_emp = three_column_final_skills.groupby('employee_id')
unique_employee_ids = three_column_final_skills.employee_id.unique()
sorted_unique_employee_ids = sorted(unique_employee_ids)
dictt = {'employee_id':[],'employee_skills':[]}
employee_skill_dataframe = pd.DataFrame(dictt)
for i in sorted_unique_employee_ids:
    skill_list = []
    for j in grp_emp.get_group(i)['employee_skills']:
        skill_list.append(j)
    employee_skill_dataframe.loc[len(employee_skill_dataframe.index)] = [i,",".join(skill_list)]  

In [75]:
employee_skill_dataframe.head()

,employee_id,employee_skills
0,AFT00047,"Manual Testing Functional Testing Testing,Pe..."
1,AFT00084,Build&Deploy DevOps Jenkins Coding & Developme...
2,AFT00099,Automation Testing Specialized Testing Python/...
3,AFT00252,Full Stack Web Technologies Intermediate Codin...
4,AFT00257,"Manual Testing Functional Testing Testing,Di..."


In [76]:
employee =  pd.merge(employee_skill_dataframe, employee, how='left', on='employee_id')
employee.shape

(57, 14)

In [77]:
employee.head()

,employee_id,employee_skills,first_name,last_name,category,department_name,designation_name,duhead_id,coehead_id,coe,name,work_location,project_type,allocation_percentage
0,AFT00047,"Manual Testing Functional Testing Testing,Pe...",Priyanka,Gurav,EMPLOYEE,Engineering,Principal Software Development Engineer-Test,"[AFT00304, AFT00640]",[AFT00640],[RELIABILITY_ENGINEERING],"[Stellus, Unitrends, Trial Scope]",[OFFSHORE],[CUSTOMER],400.0
1,AFT00084,Build&Deploy DevOps Jenkins Coding & Developme...,Samir,Joglekar,EMPLOYEE,Engineering,Associate Software Solution Architect,"[AFT00599, AFT00628]","[AFT00599, AFT00628]",[DESIGN_DEVELOPMENT],"[Marchex (Sustenance), Marchex – OCPS]","[ONSITE, OFFSHORE]",[CUSTOMER],300.0
2,AFT00099,Automation Testing Specialized Testing Python/...,Prathamesh,Sarang,EMPLOYEE,Engineering,Software Test Solution Architect,"[AFT00304, AFT00640, AFT00129]",[AFT00640],"[DESIGN_DEVELOPMENT, RELIABILITY_ENGINEERING]","[Automation training, QAccelerator, Bench, Man...",[OFFSHORE],"[ASSET, CUSTOMER]",1000.0
3,AFT00252,Full Stack Web Technologies Intermediate Codin...,Sunil,Jain,INTERN,Engineering,Principal Software Development Engineer,"[AFT00628, AFT00640]","[AFT00599, AFT00640]",[DESIGN_DEVELOPMENT],"[Convoy, Convoy- New, Convoy – EDI]",[OFFSHORE],[CUSTOMER],300.0
4,AFT00257,"Manual Testing Functional Testing Testing,Di...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
vector = cv.fit_transform(employee['employee_skills']).toarray()
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)

In [79]:
employee['designation_name'] = employee['designation_name'].fillna('no_info')
employee['coe'] = employee['coe'].fillna('no_info')
employee['category'] = employee['category'].fillna('no_info')
employee['duhead_id'] = employee['duhead_id'].fillna('no_info')
employee['coehead_id'] = employee['coehead_id'].fillna('no_info')
employee['name'] = employee['name'].fillna('no_info')
employee['work_location'] = employee['work_location'].fillna('no_info')
employee['project_type'] = employee['project_type'].fillna('no_info')

In [80]:
employee['allocation_percentage'] = employee['allocation_percentage'].fillna(0)

In [81]:
result = employee.dtypes
print(result)

employee_id               object
employee_skills           object
first_name                object
last_name                 object
category                  object
department_name           object
designation_name          object
duhead_id                 object
coehead_id                object
coe                       object
name                      object
work_location             object
project_type              object
allocation_percentage    float64
dtype: object


In [82]:
pickle.dump(employee, open('../output/employee_skill.pkl', 'wb'))
pickle.dump(similarity, open('../output/similarity.pkl', 'wb'))